In [1]:
import json

import pandas as pd
import numpy as np
df = pd.read_csv('dados/SpSafe_2022.csv', delimiter=';')
pd.set_option('display.max_columns', None)


C:\Users\gabri\AppData\Local\Temp\ipykernel_28244\1362543878.py:3: DtypeWarning: Columns (8,11,12,17,18,20,21,22,23,25,26,27,28,29,30,31,32,33,36,37,39,40,42,43,44,45,46,47,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dados/SpSafe_2022.csv', delimiter=';')


<h2 align="center">
  Código criados para localizar as cidades dos dados via latitude e longitude
</h2>


<div style = "text-align: center;">
<strong>tempo de processamento longo por causa da API da biblioteca geopy</strong>
</div>


In [2]:
# Biblioteca usada para localizar o nome da cidade via latitude e longitude
from geopy.geocoders import Nominatim
geolocal = Nominatim(user_agent="tpDados")


<p align="center">
  Convertendo o tipo da coluna cidade para objeto por questões de compatibilidade com o geopy
  
</p>

In [ ]:
df["CIDADE"] = df["CIDADE"].astype("object")

In [ ]:
for indice, linha in df.iterrows():
    if pd.isna(linha["LATITUDE"]) and pd.isna(linha["LONGITUDE"]):
        continue
    if pd.isna(linha["CIDADE"]) and pd.notna(linha["LATITUDE"]) and pd.notna(linha["LONGITUDE"]):
        try:
            lat = linha["LATITUDE"]  # Pega a latitude da linha atual
            long = linha["LONGITUDE"]  # Pega a longitude da linha atual
            local = geolocal.reverse(f"{lat}, {long}")
            
            # Tenta pegar "city", se não existir, tenta "town", se não existir, preenche com NaN
            df.at[indice, "CIDADE"] = local.raw["address"].get("city") or local.raw["address"].get("town", np.nan)
        except ValueError:
            df.at[indice, "CIDADE"] = np.nan



<p align="center">
  Comando para salvar o DataFrame em um arquivo csv
</p>

In [164]:
df.to_csv('dados/SpSafe_2022.csv', index=False, sep=';', encoding='utf-8')

In [ ]:
cache_geolocal = {}
for indice, linha in df.iterrows():
    if pd.isna(linha["LATITUDE"]) and pd.isna(linha["LONGITUDE"]):
        continue
    if pd.isna(linha["CIDADE"]) and pd.notna(linha["LATITUDE"]) and pd.notna(linha["LONGITUDE"]):
        lat_long = (linha["LATITUDE"], linha["LONGITUDE"])
        if lat_long in cache_geolocal:
            df.at[indice, "CIDADE"] = cache_geolocal[lat_long]
        else:
            try:
                local = geolocal.reverse(f"{lat_long[0]}, {lat_long[1]}")
                cidade = local.raw["address"].get("city") or local.raw["address"].get("town", np.nan)
                df.at[indice, "CIDADE"] = cidade
                cache_geolocal[lat_long] = cidade
            except ValueError:
                df.at[indice, "CIDADE"] = np.nan
                
df.to_csv('dados/SpSafe_2022.csv', index=False, sep=';', encoding='utf-8')

In [13]:
import pickle

#Salvar cache de local
with open("cache_geolocal.pkl", "wb") as f:
    pickle.dump(cache_geolocal, f)



#Salvar indice
with open("indice.pkl", "wb") as l:
    pickle.dump(indice, l)
    
